In [55]:
from gensim.models import KeyedVectors, Word2Vec
from gensim.parsing.preprocessing import remove_stopwords
from tinydb import TinyDB, Query
import numpy as np
import utils
import re
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
# Load the polish Word2Vec model
word2vec = KeyedVectors.load('./word2vec_100_3_polish.bin')

In [98]:
word2vec.most_similar(positive=['kaczyński'], topn=10)

[('kwaśniewski', 0.8283053636550903),
 ('wałęsa', 0.79841548204422),
 ('buzek', 0.7497314214706421),
 ('jaruzelski', 0.7260560393333435),
 ('macierewicz', 0.7250693440437317),
 ('kaczyńskiego', 0.7235855460166931),
 ('korfanty', 0.7205707430839539),
 ('lepper', 0.7201309204101562),
 ('cimoszewicz', 0.7198045253753662),
 ('raczkiewicz', 0.7187886238098145)]

In [57]:
db_wyborcza = TinyDB('./data/wyborcza.json')
db_tvpinfo = TinyDB('./data/tvpinfo.json')

sample_article = db_tvpinfo.all()[0]
sample_article = sample_article['content'].strip()

In [69]:
processed_sample_article = utils.preprocess(sample_article)

In [70]:
processed_sample_article

[['wicepremier',
  'jarosław',
  'kaczyński',
  'oświadczył',
  'spotkania',
  'wyborcami',
  'połajewie',
  'wielkopolsce',
  'wybory',
  'parlamentarne',
  'odbędą',
  'terminie',
  'niezależnie',
  'działo',
  'granicy',
  'białorusią',
  'związku',
  'obecnością',
  'grupy',
  'wagnera'],
 ['zapewnimy',
  'polsce',
  'bezpieczeństwo',
  'jesienią',
  'zwyciężymy',
  'podkreślił',
  'prezes',
  'prawa',
  'sprawiedliwości'],
 ['szaleństwo',
  'gorzej',
  'cyniczna',
  'wyrachowana',
  'gra',
  'przeciwko',
  'polsce',
  'wiceprezes',
  'pis'],
 ['premier', 'beata', 'szydło', 'odniosła', 'komentarza'],
 ['wicepremier',
  'jarosław',
  'kaczyński',
  'poinformował',
  'spotkania',
  'wielkopolsce',
  'poinformował',
  'niebawem',
  'ciągu',
  'kilku',
  'tygodni',
  'przedstawiony',
  'zostanie',
  'kompleksowy',
  'program',
  'wyborczy',
  'prawa',
  'sprawiedliwości'],
 ['wizja', 'tłumaczył', 'supernowoczesnego', 'państwa'],
 ['wizja', 'odzyskiwania', 'polski', 'zapowiedział', 'jar

In [100]:
nlp = spacy.load('pl_core_news_lg')

In [102]:
# Compute the mean computationally for each sentence in the text
for sentence in processed_sample_article[:1]:
    mean_v = np.zeros((100,))
    skipped = []
    for i in range(len(sentence)):
        word = sentence[i]
        word_lemma = nlp(word)[0].lemma_.lower()

        try:
            mean_v += (word2vec[word_lemma] - mean_v) / (i+1)
            print(word_lemma)
        except KeyError:
            #try:
            #mean_v += (word2vec[word] - mean_v) / (i+1)
                #print(word)
            #except KeyError:
            skipped.append(word)

    print(mean_v)

[ 1.29653482 -0.15523635 -1.01658174  1.54920832  0.6179437  -0.99526896
  1.6993542   1.38833647 -0.42245428 -1.53625358 -0.44456899 -1.08549728
  0.11063226  0.51589529  1.06173256  0.89324645  0.94144165  1.7006488
 -1.17006021 -1.64961965 -1.60179273 -0.04884334 -0.15546773 -0.72408457
  1.38530837 -1.10179012  0.39378426 -0.38087779  1.58828883  0.66802766
  1.00141519  0.6808269  -2.59260305 -1.37818378 -0.1062671  -0.15418141
 -0.00402065 -0.5389423   0.17004115  0.7554651   1.08110834  1.10038806
 -0.73524086  0.42293556 -0.73426611 -0.31433359 -1.47147792 -0.44214154
 -0.25329079 -1.08709409 -0.21785989  0.67705675  1.41615241 -0.57089845
 -1.46578408 -0.39910665 -0.6616347  -0.63119958 -1.09613299 -0.69768889
  0.60036612  1.52579771 -0.57339979 -1.36510695  0.25477191  0.58778953
 -0.26506815 -1.73583886  0.209153    0.14015815  0.64605702 -0.7246454
 -1.32321497 -1.0564635   1.67417038 -0.5352429   0.95553935  0.65172175
  1.03869116 -0.32608956  0.32388237  0.90261488  0.7

In [103]:
skipped

[]